# Gemma-2-2b-it finetuning in a local laptop using S-Group's Q&A data

This finetuning uses the unsloth technique. More information can be found from [GitHub](https://github.com/unslothai/unsloth).

In [1]:
import os
from unsloth import FastLanguageModel
import torch
from IPython.display import Markdown, display
import time
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Gemma-2-2b-it is a gated model which requires a token

In [2]:
token = os.environ.get('HUGGINGFACE_TOKEN')

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-2-2b-it",
    max_seq_length = max_seq_length,
    dtype = dtype,
    token=token,
)

==((====))==  Unsloth 2025.1.8: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4090 Laptop GPU. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Inference without finetuning
We first investigate the inference before finetuning and after finetuning we ask the same questions.
The unsloth model uses the alpaca_prompt.

In [4]:
alpaca_prompt = """Käytät vain suomen kieltä vastauksissa. Alla on OHJE, joka kuvaa tehtävän. SYÖTE on lisäkontekstia tehtävään. Kirjoita VASTAUS, joka täydentää pyyntöä asianmukaisesti.

### OHJE:
{}

### SYÖTE:
{}

### VASTAUS:
{}"""

In [5]:
FastLanguageModel.for_inference(model) 

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layerno

In [6]:
def generate_response(instruction: str) -> str:
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instruction,
                "",
                "",
            )
        ],
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        use_cache=True,
        repetition_penalty=1.2,
        no_repeat_ngram_size=4,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
    )
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return decoded_output

In [7]:
display(Markdown(generate_response("Miten voin nähdä minusta kerätyt tiedot?")))

Käytät vain suomen kieltä vastauksissa. Alla on OHJE, joka kuvaa tehtävän. SYÖTE on lisäkontekstia tehtävään. Kirjoita VASTAUS, joka täydentää pyyntöä asianmukaisesti.

### OHJE:
Miten voin nähdä minusta kerätyt tiedot?

### SYÖTE:


### VASTAUS:
Tiedon näyttöä voi tarkistaa seuraavasti: 

* **[Linkki tai ohjeet]** - Näitä kautta voit katsoa tietoja, jotka olet kerätty käyttämällä [tietojen luokittelu]. 
* **Yhteydenotto** - Voit ottaa yhteyttä [henkilö] ja kysyä lisää tietoa.





In [8]:
display(Markdown(generate_response("Mikä on S-Tunnus?")))

Käytät vain suomen kieltä vastauksissa. Alla on OHJE, joka kuvaa tehtävän. SYÖTE on lisäkontekstia tehtävään. Kirjoita VASTAUS, joka täydentää pyyntöä asianmukaisesti.

### OHJE:
Mikä on S-Tunnus?

### SYÖTE:


### VASTAUS:
S-tunnus on Suomen pankin ja rahoitusyritysten käyttämä tunnukset numero. Se on 10-saksinen numerokirjaimella ja numeroilla varustettu tunnusnumero.  





In [9]:
display(Markdown(generate_response("Kuinka voin kieltää markkinoinnin?")))

Käytät vain suomen kieltä vastauksissa. Alla on OHJE, joka kuvaa tehtävän. SYÖTE on lisäkontekstia tehtävään. Kirjoita VASTAUS, joka täydentää pyyntöä asianmukaisesti.

### OHJE:
Kuinka voin kieltää markkinoinnin?

### SYÖTE:


### VASTAUS:
Markkinoiden kielenkäyttöön liittyvät kysymykset ovat monimutkaisia ja riippuvat useista tekijöistä. Tässä muutamia yleisiä vinkkejä, jotka auttavat sinua kieltämään markkinointi:** 

* **Ymmärrä markkinoinnille tarkoitettujen viestien sisältö.** Tarkistathan, mitä mainos- tai markkinointimateriaalia haluat kieltää. Onko se teksti, video, banner vai jotain muu? Yrittä ymmärtää, miksi tämä markkinointi on tärkeää yrityksen kannalta.
* **Huomaa markkinoinnilla käytettävät tekniikat.**  Mitä mainonnan ohjaavat? Mitä tavoittelee? Miten se vaikuttaa käyttäjien ajatteluun? Näitä asioita kannattaa tarkastella ennen kuin olet varma siitä, että haluaisit kieltää sen.
*  **Tutki markkinoinninhallintaa.** Mikä on oikeus käyttää markkinointia? Kuvastaatko sitä viranomaisten säädöksillä? Voit hakea tietoa näiden aiheista paikallisilta viranomaisilta.
*   **Tarkasta lainsäädäntöä.** Markkinoinnin kieltämiseen voi olla olemassa lakia. Tutustu siihen ja varmista, että olet tietoinen kaikesta.
*    **Varmista, että kieleen ei ole yhteyttä.** Jos markkinointi ei ole vaikeaa, voit kokeilla kieltää sitä. Muista kuitenkin, että kaikki asiakkaat eivät välttämättä hyväksyvät kieltämistä.



On tärkeää muistaa, että kielen kieltäminen on yksilöllinen prosessi, josta tulee paljon kokemusta. Se edellyttää aikaa ja ponnistelua. 


## Finetuning setup

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


## Data Preparation

We now use the S-Group Q&A data from [S-Kanava](https://www.s-kanava.fi/asiakaspalvelu/) and [Data Protection](https://tietosuoja.s-ryhma.fi/ukk/) page. These are together only 248 rows which is a very small dataset.

The unsloch finetuning uses the alpaca_prompt format.

The dataset is loaded from the local disk.

In [11]:
import json
from datasets import Dataset

In [12]:
def load_json_dataset(json_file_path):
    """
    Loads a dataset from a local JSON file.

    Args:
        json_file_path (str): The path to the JSON file.

    Returns:
        datasets.Dataset: A Hugging Face Dataset object.
    """
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Transform the data to a dictionary of lists
        transformed_data = {
            "tulos": [row["tulos"] for row in data],
            "syöte": [row["syöte"] for row in data],
            "ohje": [row["ohje"] for row in data]
        }

        # Create a Hugging Face Dataset from the transformed data
        dataset = Dataset.from_dict(transformed_data)
        return dataset

    except FileNotFoundError:
        print(f"Error: JSON file not found at '{json_file_path}'")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [13]:
json_file = "input.json" 
dataset = load_json_dataset(json_file)

In [14]:
print(dataset)

Dataset({
    features: ['tulos', 'syöte', 'ohje'],
    num_rows: 248
})


Dataset is formatted for finetuning.

In [15]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["ohje"]
    inputs       = examples["syöte"]
    outputs      = examples["tulos"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

## Train the model
We use the Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 120 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
   #     num_train_epochs = 3, # Set this for 1 full training run.
        max_steps = 120,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/248 [00:00<?, ? examples/s]

In [17]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090 Laptop GPU. Max memory = 15.992 GB.
2.668 GB of memory reserved.


In [18]:
start_time = time.time()
trainer_stats = trainer.train()
end_time = time.time()
print(f"Finetuning took {end_time - start_time:.2f} seconds.")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 248 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 120
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,2.787900
2,2.999100
3,2.825900
4,2.747300
5,2.805500
6,2.580300
7,2.753800
8,2.495500
9,2.307500
10,2.318000


Finetuning took 198.67 seconds.


In [20]:
print(f"Finetuning took {end_time - start_time:.2f} seconds.")

Finetuning took 198.67 seconds.


In [21]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

197.6762 seconds used for training.
3.29 minutes used for training.
Peak reserved memory = 4.18 GB.
Peak reserved memory for training = 1.512 GB.
Peak reserved memory % of max memory = 26.138 %.
Peak reserved memory for training % of max memory = 9.455 %.


<a name="Inference"></a>
### Inference
Let's run the model. You can change the instruction and input - leave the output blank.

In [22]:
FastLanguageModel.for_inference(model) 

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [23]:
def generate_response(instruction: str) -> str:
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instruction,
                "",
                "",
            )
        ],
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        use_cache=True,
        repetition_penalty=1.2,
        no_repeat_ngram_size=4,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
    )
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return decoded_output

In [24]:
display(Markdown(generate_response("Miten voin nähdä minusta kerätyt tiedot?")))

Käytät vain suomen kieltä vastauksissa. Alla on OHJE, joka kuvaa tehtävän. SYÖTE on lisäkontekstia tehtävään. Kirjoita VASTAUS, joka täydentää pyyntöä asianmukaisesti.

### OHJE:
Miten voin nähdä minusta kerätyt tiedot?

### SYÖTE:


### VASTAUS:
Voit tarkistaa ja muokata tietojasi S-käyttäjätilillä osoitteessa s-kayttajatili.fi tai soittamalla asiakasomistajapalveluun puh. 010 76 5858 ma–pe 9–16 (mpm/pvm).

In [25]:
display(Markdown(generate_response("Mikä on S-Tunnus?")))

Käytät vain suomen kieltä vastauksissa. Alla on OHJE, joka kuvaa tehtävän. SYÖTE on lisäkontekstia tehtävään. Kirjoita VASTAUS, joka täydentää pyyntöä asianmukaisesti.

### OHJE:
Mikä on S-Tunnus?

### SYÖTE:


### VASTAUS:
S-Tunnusta käytetään digitaalisen asiakasrekisterin tunnistautumiseen ja verkkopalveluiden käyttöönottoon sekä muihin yhteistyökumppaneille tarkoitettuihin palveluihin.

In [26]:
display(Markdown(generate_response("Kuinka voin kieltää markkinoinnin?")))

Käytät vain suomen kieltä vastauksissa. Alla on OHJE, joka kuvaa tehtävän. SYÖTE on lisäkontekstia tehtävään. Kirjoita VASTAUS, joka täydentää pyyntöä asianmukaisesti.

### OHJE:
Kuinka voin kieltää markkinoinnin?

### SYÖTE:


### VASTAUS:
Voit halutessasi antaa meille kielto tai muuttaa tietojasi S-käyttäjätilillä ja asiakasomistajatalouksiisi liittyessäsi osoitteessa s-kayttajatili.fi . Voit myös ilmoittaa tietojesi poistamisesta sähköpostilla osoitteella palaute@sok.fi (palautelomake)

## Saving finetuned model
To save the final model as LoRA adapters, use Huggingface's `push_to_hub` for an online save.

In [27]:
upload_token = os.environ.get('UPLOAD_TOKEN')

In [29]:
model.push_to_hub_merged("gemma-2-2b-it-S-Group", tokenizer, save_method = "merged_16bit", token = upload_token)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.53 out of 15.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 39.05it/s]


Unsloth: Saving to organization with address mlconvexai/gemma-2-2b-it-S-Group
Unsloth: Saving tokenizer... Done.
Unsloth: Saving to organization with address mlconvexai/gemma-2-2b-it-S-Group
Unsloth: Uploading all files... Please wait...


  0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/gemma-2-2b-it-S-Group
